# Infos about the simulation class

# Libraries

In [4]:
import numpy as np
import numpy.random as rnd
import numpy.linalg
import matplotlib.pyplot as plt
from numpy import linalg as LA
from numpy.random import normal
import scipy
from scipy.spatial.transform import Rotation as R
from pathlib import Path

In [5]:


class Configuration:
    #"""generate a configuration of spins"""
    
    def __init__(self, a,theta,Nx,Ny, J,seed_control=False,seed=1500):
        #'we create a kagome lattice from parameter a and theta with Nx*Ny*3 spins , '
        self.a,self.theta,self.Nx,self.Ny,self.J=a,theta,Nx,Ny,J,
        self.N=Nx*Ny*3

        # a_1 and a_2 are the lattice translation generating vectors 
        a_1=a*np.array([1,0])
        a_2=a*np.array([-1*np.cos(theta),np.sin(theta)])
        
        # we construct the unit cell
        unit_cell=np.array([[0,0],0.5*a_2,0.5*a_1])
        
        # we construct the grid points from the translations of the lattice
        self.x_l,self.y_l=np.meshgrid(range(Nx),range(Ny))
        A_1=np.tensordot(self.y_l,[a_1,a_1,a_1],axes=0)
        A_2=np.tensordot(self.x_l,[a_2,a_2,a_2],axes=0)

        # the lattice points coordinates are created from adding a unit cell at each grid point
        self.lattice=np.tensordot(np.ones((Nx,Ny)),unit_cell,axes=0)+A_1+A_2

        # we create a dim object containing the dimensions of the lattice 
        dim=self.lattice.shape

        #seed control for when we want to test with the same configuration
        if seed_control==True:
            np.random.seed(seed) 

        # then we create a random spin configuration  self.config as a unit 3d vector on each grid point
        self.config=2*np.random.rand(dim[0],dim[1],dim[2],3)-1*np.ones((dim[0],dim[1],dim[2],3))

        # we make sure they are unit vectors
        Norm=np.repeat(LA.norm(self.config,axis=3)[:, :,:, np.newaxis], 3, axis=3)
        self.config/=Norm
        
        # we want to save some value while the simulation runs for post processing
        self.Energy=[self.total_energy()]
        self.acceptation_rates=[]
        #self.angles=[]
        #self.overrelaxation_dose=[]
        self.Capa=[]
        #self.counter=0

    def get_mean_field(self):
        """this part implements the sommation over spins nearest spins"""
        self.mean_field=np.zeros(self.config.shape)
        self.mean_field[:,:,0,:]=self.config[:,:,1,:]+self.config[:,:,2,:]
        self.mean_field[:,:,1,:]=self.config[:,:,2,:]+self.config[:,:,0,:]
        self.mean_field[:,:,2,:]=self.config[:,:,0,:]+self.config[:,:,1,:]


        #self.mean_field[:,:,0,:]+= np.roll(self.config[:,:,1,:],(0,-1),axis=(0,1)).copy()+np.roll(self.config[:,:,2,:],(-1,0),axis=(0,1)).copy()
        #self.mean_field[:,:,1,:]+=np.roll(self.config[:,:,2,:],(-1,1),axis=(0,1)).copy()+np.roll(self.config[:,:,0,:],(0,1),axis=(0,1)).copy()
        #self.mean_field[:,:,2,:]+=np.roll(self.config[:,:,0,:],(1,0),axis=(0,1)).copy()+np.roll(self.config[:,:,1,:],(1,-1),axis=(0,1)).copy()


        self.mean_field[:,:,0,:]+= np.roll(self.config[:,:,1,:],(0,1),axis=(0,1)).copy()+np.roll(self.config[:,:,2,:],(1,0),axis=(0,1)).copy()
        self.mean_field[:,:,1,:]+=np.roll(self.config[:,:,2,:],(1,-1),axis=(0,1)).copy()+np.roll(self.config[:,:,0,:],(0,-1),axis=(0,1)).copy()
        self.mean_field[:,:,2,:]+=np.roll(self.config[:,:,0,:],(-1,0),axis=(0,1)).copy()+np.roll(self.config[:,:,1,:],(-1,1),axis=(0,1)).copy()



        return self.mean_field
  


    def total_energy(self):
        M1=self.get_mean_field()
        M2=self.config

        #print('mean',B)
        #print('config',self.config)
        #M1=np.reshape(B,(3*self.Nx*self.Ny,3),order='C')
        #M2=np.reshape(self.config,(3*self.Nx*self.Ny,3),order='C')


        #E=self.J*np.linalg.tensordot(M1,M2,axes=2)
        #E=self.J*np.sum(np.einsum('ij,ij->i', M1, M2))
        E=self.J*np.sum(np.einsum('ijkl,ijkl->ijk', M1, M2))
        #print('energy',E)
        return E





    def delta_energy(self):
        ''' this function is used to compute the energy difference for a flip '''
        M1=self.get_mean_field()
        M2=self.flipped_spin-self.config
        
        #M1=np.reshape(B,(3*self.Nx*self.Ny,3),order='C').copy()
        #print(M1.shape)
        #M2=np.reshape(self.flipped_spin-self.config,(3*self.Nx*self.Ny,3),order='C').copy()
        #print(M2.shape)
        #print(np.einsum('ij,ij->i', M1, M2).shape)
        C=np.einsum('ijkl,ijkl->ijk', M1, M2)

        #C=np.reshape(C,(self.Nx,self.Ny,3),order='C')
        #print('Cshape',C)

        #we compute the difference in energy contribution between the flipped and unaltered spin
        Delta_E=(self.J)*C
        #print('delta',Delta_E)
        
        return Delta_E
    def overrelaxation_2(self,dose=0.1):
        '''In this part we implement Nf iterrations of overrelaxation'''
      
        #self.angles.append(angles_siz
        #self.overrelaxation_dose.append(Nf)
        # we pick random angles for rotation around the mean field angles_size paramter controling how much we rotate
        #a = np.pi*(2*np.random.rand(Nf)-np.ones(Nf))*angles_size

        # we select the number Nf of sites that will be rotated
        Nf=int(round(dose*self.N))
        #flips=np.random.randint([0,0,0],[self.Nx,self.Ny,3],size=(Nf,3))
        B=self.get_mean_field()
        Norm=np.repeat(LA.norm(B,axis=3)[:, :,:, np.newaxis], 3, axis=3)
        B/=Norm

        #angle=np.pi
        pick = np.zeros(self.N)
        pick[:Nf-1]=1
        #pick=np.concatenate((np.ones(Nf),np.zeros(self.N-Nf)))
        #print(pick.shape,pick)

        np.random.shuffle(pick)
        #print(pick.shape,pick)
        pick=np.reshape(pick,(self.Nx,self.Ny,3))
        #print(pick.shape,pick)
        pick=pick.astype(bool)

        newpick=np.repeat(pick[:, :,:, np.newaxis], 3, axis=3)
        #print(pick.shape,pick)
        #pick=np.reshape(pick,(self.Nx,self.Ny,3))
        #decision=np.where(pick)
        #print(len(decision),len(decision[0]))

        dotproduct=np.einsum('ijkl,ijkl->ijk', B, self.config)
        newdotproduct=np.repeat(dotproduct[:, :,:, np.newaxis], 3, axis=3)
        normal=self.config-newdotproduct*B



        newarr=self.config-2*normal

        self.config=np.where(newpick,newarr,self.config )
        #print('norm=',LA.norm(self.config,axis=3).max())


    def overrelaxation(self,dose=0.1):
        '''In this part we implement Nf iterrations of overrelaxation'''
      
        #self.angles.append(angles_size)
        #self.overrelaxation_dose.append(Nf)
        # we pick random angles for rotation around the mean field angles_size paramter controling how much we rotate
        #a = np.pi*(2*np.random.rand(Nf)-np.ones(Nf))*angles_size

        # we select the number Nf of sites that will be rotated
        Nf=int(round(dose*self.N))
        #flips=np.random.randint([0,0,0],[self.Nx,self.Ny,3],size=(Nf,3))
        B=self.get_mean_field()

        angle=np.pi
        
        pick=np.concatenate((np.ones(Nf),np.zeros(self.N-Nf)))
        #print(pick.shape,pick)

        np.random.shuffle(pick)
        #print(pick.shape,pick)
        pick=np.reshape(pick,(self.Nx,self.Ny,3))
        #print(pick.shape,pick)
        pick=pick.astype(bool)

        newpick=np.repeat(pick[:, :,:, np.newaxis], 3, axis=3)
        #print(pick.shape,pick)
        #pick=np.reshape(pick,(self.Nx,self.Ny,3))
        #decision=np.where(pick)
        #print(len(decision),len(decision[0]))
        Rot=np.zeros((self.Nx,self.Ny,3,4))
        #print(B.shape)
        Rot[:,:,:,0]=np.cos(angle/2)
        #print(B[decision,:].shape)
        #print(self.config[decision,:].shape)
        
        Rot[:,:,:,1:]=np.sin(angle/2)*B
        Norm=np.repeat(LA.norm(Rot,axis=3)[:, :,:, np.newaxis], 4, axis=3)
        Rot/=Norm
        Rotad=np.reshape(Rot,(self.N,4),order='C').copy()

        #Norm=LA.norm(Rot,axis=3)
        r = R.from_quat(Rotad,scalar_first=True)

        newarr=r.apply(  np.reshape(self.config,(self.N,3),order='C').copy())
        self.config=np.where(newpick,np.reshape(newarr,(self.Nx,self.Ny,3,3),order='C') ,self.config )




    def Monte_Carlo(self,Nf,T):
        #'''In this part we implement the Nf monte carlo steps
        #here with the metropolis move'''

        #self.verify_norm()
        beta=1/T
        #self.energies=[]
        for comp in range(Nf):
            #mean=self.get_mean_field
            

                    
            n = np.random.normal(0,(3*T,3*T,T),(self.Nx,self.Ny,3,3))

            self.flipped_spin=self.config+n

            Norm=np.repeat(LA.norm(self.flipped_spin,axis=3)[:, :,:, np.newaxis], 3, axis=3)
            self.flipped_spin/=Norm
            

            delta=self.delta_energy()

            #proba=np.exp(-beta*delta)
            #alea=np.random.rand(self.Nx,self.Ny,3)
            proba=np.random.rand(self.Nx,self.Ny,3)
            expener=np.exp(-beta*delta)
            decision=np.where(proba< expener)
            #print('decision',len(decision),len(decision[0]))
            #print('old',self.config[decision][:])
            #self.config[decision,:]=self.flipped_spin[decision,:]

            newproba = np.repeat(proba[:, :,:, np.newaxis], 3, axis=3)
            newexpener= np.repeat(expener[:, :,:, np.newaxis], 3, axis=3)
            self.config=np.where(newproba<= newexpener,self.flipped_spin,self.config)



            #print(newproba.shape,newproba[:,:,:,0]==newproba[:,:,:,2])
            
            #self.config[:,:,:,0]=np.where(proba<= expener,self.flipped_spin[:,:,:,0],self.config[:,:,:,0])
            #self.config[:,:,:,1]=np.where(proba<= expener,self.flipped_spin[:,:,:,1],self.config[:,:,:,1])
            #self.config[:,:,:,2]=np.where(proba<= expener,self.flipped_spin[:,:,:,2],self.config[:,:,:,2])
            #print('delta',delta[decision])
            #print('flipped',self.flipped_spin[decision][:])

            #print('new',self.config[decision][:])
            #newspin=np.reshape(self.flipped_spin,(3*self.Nx*self.Ny,3),order='C').copy()
            #configspin=np.reshape(self.config,(3*self.Nx*self.Ny,3),order='C').copy()
            #proba=np.random.rand(self.Nx*self.Ny*3)
            #print('norm=',LA.norm(self.config,axis=0).max())
            #('normf=',LA.norm(self.flipped_spin,axis=0).max())

            #decision=np.argwhere(proba< expener)
            #print(decision.shape)
            flag=len(decision[0])
            #self.config/=LA.norm(self.config,axis=3)
            #configspin[decision][:]=newspin[decision][:]
            
            #self.config=np.reshape(configspin,(self.Nx,self.Ny,3,3),order='C').copy()
            
            self.overrelaxation_2()
            #self.verify_norm()




    def measure_Capa(self,Nf,T):
        beta=1/T
        #self.energies=[]
        Ener=[]
        for comp in range(Nf):
            #mean=self.get_mean_field


                    
            n = np.random.normal(0,(3*T,3*T,T),(self.Nx,self.Ny,3,3))

            self.flipped_spin=self.config+n
            #print('flippedold',LA.norm(self.flipped_spin,axis=0).max())
            Norm=np.repeat(LA.norm(self.flipped_spin,axis=3)[:, :,:, np.newaxis], 3, axis=3)
            self.flipped_spin/=Norm
            #print('flippenew',LA.norm(self.flipped_spin,axis=0).max())
            delta=self.delta_energy()

            #proba=np.exp(-beta*delta)
            #alea=np.random.rand(self.Nx,self.Ny,3)
            proba=np.random.rand(self.Nx,self.Ny,3)
            expener=np.exp(-beta*delta)
            decision=np.where(proba<= expener)

            newproba = np.repeat(proba[:, :,:, np.newaxis], 3, axis=3)
            newexpener= np.repeat(expener[:, :,:, np.newaxis], 3, axis=3)
            #print(np.array_equal(np.where(newproba[:,:,:,0]<=newexpener[:,:,:,0]), np.where(newproba[:,:,:,1]<=newexpener[:,:,:,1])))
            self.config=np.where(newproba<= newexpener,self.flipped_spin,self.config)
            #.config[decision,:]=self.flipped_spin[decision,:]
            #newspin=np.reshape(self.flipped_spin,(3*self.Nx*self.Ny,3),order='C').copy()
            #configspin=np.reshape(self.config,(3*self.Nx*self.Ny,3),order='C').copy()
            #proba=np.random.rand(self.Nx*self.Ny*3)
            
            #print('afterdec',LA.norm(self.config,axis=0).max())
            #decision=np.argwhere(proba< expener)
            #print(decision.shape)
            flag=len(decision[0])

            #configspin[decision][:]=newspin[decision][:]
            
            #self.config=np.reshape(configspin,(self.Nx,self.Ny,3,3),order='C').copy()

            self.overrelaxation_2()
            #self.config/=LA.norm(self.config,axis=3)




            if comp%100==0:
                Ener.append(self.total_energy())
                print(self.total_energy())
                self.acceptation_rates.append(flag/self.N)
                print(flag/self.N,T)
                
                
        #print(Ener)
        #print(1/T)
        Capacite=np.var(Ener)/(T**2)
        self.Capa.append(Capacite)
        self.Energy=np.concatenate((self.Energy,Ener))



    def verify_norm(self):
        """ we have sometimes a problem with the  spin vectors not 
        being normalised after too many rotations due to errors so we normalise them after each 
        flip and we verify their maximum norms"""
        print('norm=',LA.norm(self.config,axis=3).max())
        print('normf=',LA.norm(self.flipped_spin,axis=3).max())

    def saveconfig(self,title,T):
        # code for saving the spin configuration with parameters
        mydict={
        "Temperature": T,
        "Nx": self.Nx,
        "Ny": self.Ny,
        "J": self.J,
        "N":self.N,
        "lattice":np.array(self.lattice),
        "configuration":np.array(self.config),
        "Energy":np.array(self.Energy),
        "acceptation_rate":np.array(self.acceptation_rates)  ,
        "Capacité":np.array(self.Capa)  ,
        }
        #np.save(title+'infos',[T,self.Nx,self.Ny,J,self.N])
        np.save(title, mydict)



In [ ]:

#parameters of simulation
J=1
kb=1.38*10**-23
#T=0.2
Ntemp=50
Temp=np.concatenate((np.power(10*np.ones(10),np.linspace(-0.1,-1.5,10)),np.power(10*np.ones(30),np.linspace(-1.501,-3,30))))
#Temp=[0.2]
print(Temp)
L=5
Nx=L
Ny=L
theta=2*np.pi/3
a=1
N=Nx*Ny*3

#initialise simulation
A=Configuration(1,theta,Nx,Ny,J)
#print(A.delta_energy())
#print(A.total_energy2())
print(A.total_energy())
#print(A.total_energy2()/A.total_energy())
#print((A.total_energy())/N)

parameters_granularity=4
angles_sizes_arr=np.array([1,1,1,0.5])
overrelaxation_dose=0*np.array([500,1000,1000,1000])
#tuning_parameters=[[5,0],[2,0],[1,0],[0.8,100],[0.7,1000],[0.5,1000],[0.4,1000]]

#print(tuning_parameters[0][0])
#Energies=[]
acceptationrate=[0.5]
#Total_of_flipps=1*10**5



for j,T in enumerate(Temp):
    
    
    #while flipped<=Total_of_flipps:
    decoherencetime=10000
    numberMC=10000
   
    A.Monte_Carlo(decoherencetime,T)
    A.verify_norm()
        #c_rate,numberfliped=A.Monte_Carlo(1,T)
    for q in range(5):
        A.measure_Capa(numberMC,T)
        A.verify_norm()
        #A.overrelaxation(20)
        A.verify_norm()
    print(j,'done')

    #acceptationrate.append(c_rate)
    #flipped+=numberfliped+overrelaxation_dose[p1]


    #print(c_rate,p1)
    #if min(acceptationrate)<=0.2:
     #   p1=p1+1
     #   acceptationrate=[0.5]
    #if max(acceptationrate)>=0.8:
       # p1=p1-1
       # acceptationrate=[0.5]
       
        #p1=np.clip(p1,0,parameters_granularity-1)

    #print('we flipped',flipped,'  spins')
    #dirname = os.path.dirname('/Results/testL'+str(L)+str(j))
    #title = os.path.join(dirname, '/Results/testL'+str(L)+str(j))
    #title=os.getcwd()-'/Design'+'/Results/testL'+str(L)+str(j)
    title='/users/eleves-b/2025/tanoh.nguessan/Downloads/github/Numerical-Physics-Project-order-by-disorder-/Results/testnewoverL'+str(L)+str(j)
    #template_path = Path(__file__).parent.parent / 'Results' / 'testL'+str(L)+str(j)+'.npy'
    #title=template_path # This will give you the correct path to the template
    
    A.saveconfig(title,T)





[0.79432823 0.55519359 0.38805107 0.27122726 0.18957357 0.13250194
 0.09261187 0.06473082 0.04524343 0.03162278 0.03155005 0.02800982
 0.02486685 0.02207654 0.01959934 0.0174001  0.01544764 0.01371426
 0.01217539 0.01080919 0.00959629 0.00851949 0.00756352 0.00671482
 0.00596135 0.00529243 0.00469857 0.00417134 0.00370328 0.00328773
 0.00291882 0.0025913  0.00230053 0.00204239 0.00181321 0.00160975
 0.00142912 0.00126876 0.00112639 0.001     ]
6.4714485306753975
norm= 1.0000000000000002
normf= 1.0000000000000002
-55.109282147156094
0.6133333333333333 0.7943282347242815
-69.54836122628936
0.5466666666666666 0.7943282347242815
-60.160439942983714
0.56 0.7943282347242815
-84.07859724736169
0.5066666666666667 0.7943282347242815
-55.845829367917
0.56 0.7943282347242815
-53.56552135759348
0.6666666666666666 0.7943282347242815
-54.858466451154605
0.6666666666666666 0.7943282347242815
-38.81120369680669
0.6666666666666666 0.7943282347242815
-66.38481961808233
0.52 0.7943282347242815
-69.835845